In [58]:
!pip install langchain

In [59]:
from langchain.llms import GooglePalm

In [60]:
!pip install -U -q google.generativeai
!pip install -q chromadb

In [61]:
# creating an object for our LLM

api_key = 'your_api_key'

llm = GooglePalm(google_api_key = api_key, temperature = 0.2)

poem = llm("write a poem on my love for ramen")
print(poem)

**Ode to Ramen**

Ramen, oh ramen,
You are my favorite food.
Your broth is so flavorful,
And your noodles are so chewy.

I love the way you make me feel warm and cozy,
Like I'm wrapped in a blanket on a cold winter day.
You're the perfect comfort food,
And I can't get enough of you.

I love the way you smell,
So inviting and delicious.
And I love the way you taste,
So umami and satisfying.

Ramen, oh ramen,
You are my one true love.
I will always cherish you,
And I will never stop eating you.

You are the best food in the world,
And I will never forget you.

Thank you for being my favorite food,
Ramen.


In [62]:
!pip install pymysql

In [63]:
!pip install mysql-connector-python

In [64]:
!pip install langchain_experimental

In [65]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [66]:
import pymysql
import mysql.connector

import sqlalchemy as sal

from sqlalchemy import create_engine, MetaData, Table

import pyodbc

import os
import pandas as pd

In [67]:
db_host = os.getenv('MYSQL_HOST', 'localhost')
db_port = os.getenv('MYSQL_PORT', 3306)
db_user = os.getenv('MYSQL_USER', 'root')
db_password = os.getenv('MYSQL_PASSWORD', 'root')
db_name = os.getenv('MYSQL_DATABASE', 'atliq_tshirts')

In [68]:
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

In [69]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

# Creating an SQLDatabaseChain

In [71]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose = True)
question1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(79,)]
Answer:79
> Finished chain.


In [72]:
question1

'79'

In [77]:
# a complex query
question2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('360'),)]
Answer:360
> Finished chain.


In [78]:
# storing the correct query
question2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S'
SQLResult: [(Decimal('19358'),)]
Answer:19358
> Finished chain.


In [80]:
# a more complex query
# throws error
question3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounts.start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [81]:
# storing the correct query

sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

question3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('23996.000000'),)]
Answer:23996.0
> Finished chain.


In [93]:
# wrong
question4 = db_chain.run("If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('533'),)]
Answer:533
> Finished chain.


In [94]:
question4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('23996'),)]
Answer:23996
> Finished chain.


In [91]:
# wrong
question5 = db_chain.run("How many white color Levi's t shirts are available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts are available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(28,), (48,)]
Answer:48
> Finished chain.


In [92]:
question5 = db_chain.run("SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White'
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White'
SQLResult: [(Decimal('76'),)]
Answer:76
> Finished chain.


# FEW SHOT LEARNING

In [97]:
few_shots = [
    {
        'Question' : "How many t-shirts do we have left for nike in extra small size and white color?",
        'SQLQuery' : "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
        'SQLResult' : "Result of the SQL query",
        'Answer' : question1
    },
    {
        'Question' : "How much is the price of the inventory for all small size t-shirts?",
        'SQLQuery' : "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S'",
        'SQLResult' : "Result of the SQL query",
        'Answer' : question2
    },
    {
        'Question' : "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?",
        'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
        'SQLResult' : "Result of the SQL query",
        'Answer' : question3
    },
    {
        'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?",
        'SQLQuery' : "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
        'SQLResult' : "Result of the SQL query",
        'Answer' : question4
    },
    {
        'Question' : "How many white color Levi's t shirts are available?",
        'SQLQuery' : "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White'",
        'SQLResult' : "Result of the SQL query",
        'Answer' : question5
    }
]

In [99]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for torch>=1.6.0 from https://files.pythonhosted.org/packages/1e/86/477ec85bf1f122931f00a2f3889ed9322c091497415a563291ffc119dacc/torch-2.1.2-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/ef/a2/f16cac894c4c71585b3411707502ed8d607945fb4a695857621565bd728d/torchvision-0.16.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 31.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.2 MB/s eta 0:00:0000:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=e8765d1fa8ed9e5f81e61d95f02db91d2fb1cb7df73c9fd6e083

In [100]:
# generating embeddings

from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [102]:
# merging few_shots as a string

to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for nike in extra small size and white color? SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 79",
 "How much is the price of the inventory for all small size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S' Result of the SQL query 19358",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 23996.0",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' R

In [103]:
# creating the vector database

from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embedding = embeddings, metadatas = few_shots)

In [108]:
# example selector

from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(vectorstore = vectorstore, k = 2)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '79',
  'Question': 'How many t-shirts do we have left for nike in extra small size and white color?',
  'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '76',
  'Question': "How many white color Levi's t shirts are available?",
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White'",
  'SQLResult': 'Result of the SQL query'}]

# Instructor MySQL_Prompt from Lanchain

In [110]:
# custom mysql prompt that instructs the llm

from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)
print(PROMPT_SUFFIX)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

# Creating Prompt Templates

In [111]:
# normal prompt template

from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [113]:
# few-shot prompt template

from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

# Creating a new SQL DB Chain that accounts for the few-shots as well

In [114]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose = True, prompt = few_shot_prompt)

In [116]:
# correct question5
new_chain("How many white color Levi's t shirts are available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts are available?
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White'
SQLResult: [(Decimal('76'),)]
Answer:76
> Finished chain.


{'query': "How many white color Levi's t shirts are available?",
 'result': '76'}

In [118]:
# correct question2
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='S'
SQLResult: [(Decimal('19358'),)]
Answer:19358
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '19358'}

In [120]:
# new query
# correct
new_chain("How much is the price of all the extra small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all the extra small size t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size='XS'
SQLResult: [(Decimal('19843'),)]
Answer:19843
> Finished chain.


{'query': 'How much is the price of all the extra small size t-shirts?',
 'result': '19843'}

In [121]:
# a complex query
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('21744.250000'),)]
Answer:21744.25
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '21744.25'}

In [122]:
# complex query
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('25154.600000'),)]
Answer:25154.6
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '25154.6'}

In [123]:
# complex query
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('25934'),)]
Answer:25934
> Finished chain.


'25934'